# 【補足】 生成モデルによる異常部位検知

## 異常検知モデルの学習方法

異常検知モデルの学習方法には、正常データのみ学習する方法と正常データと不良データを学習する方法の2つがあります。それぞれ適用するアルゴリズムが全く異なるものになります。

|**学習データ**|**モデル**|**適用アルゴリズム例**|
|---|---|---|
|正常データのみ|生成モデル|AE, VAE, GAN|
|正常データと不良データ|識別モデル|CNN|

各状況に応じて、どのモデルを使うのかを決定する必要がありますが、豊富にデータを準備できているなら、VGG16 や ResNet に代表される事前学習済みの識別モデルを使用し、不良データが十分に無いと判断すれば生成モデルを使用します。

## 生成モデル

各種アルゴリズムを学習していく前に、生成モデルとは何なのかを学習しておく必要があります。まず、抑えておきたいのは生成モデルと識別モデルは対のプロセスだということです。

![08](img/08.png)

識別モデルは、入力があらかじめ指定されたクラスの中で最も当てはまる確率が高いものを出力にします。

生成モデルでは、上図のように、入力と出力が同じになるようにニューラルネットワークを学習させるのです。

まず Encoder と呼ばれるネットワークを通じて、潜在変数を抽出します。ここでの潜在変数とは入力変数を要約したもの（圧縮された特徴）です。例えば、花の種別を分類したいときに入力変数として「花弁の長さ」「ガクの長さ」などがあるとすると新たに「大きさ」という抽象的な尺度で表現し直すことを指します。入力変数は具体的な値で特定できることに対し、潜在変数は抽象的な表現というのがポイントです。

そして Decoder では、Encoder で抽出された潜在変数を用い、入力データを復元していきます。

学習フェーズでは正常データのみを用い、入力データを綺麗に復元できるように入力と出力との間に生じる再構成誤差 (Reconstruction Error) を少なくするように重みを更新します。推論フェーズでは生成モデル「学習していないことは復元できない」性質を用いて、入力データに傷や歪みなどの学習されてない情報があれば綺麗に復元できないようになっています。その結果、出力と入力の差分を確認して、綺麗に復元されていれば正常、復元されていない部位があれば異常と判断します。

ここまでが基本的な生成モデルを用いた異常検知の流れです。

Step1 : データを用意する  
Step2 : 正常データのみを使い、学習する  
Step3 : 学習済みモデルを使い、推論する  
Step4 : 推論結果から異常か正常か判断する  

学習時に人が確認する際には、実際に復元された出力と入力の差分を目で見て確認しますが、実運用時は、モデルが推論時に返す誤差の大小で判断します。こちらで定めた閾値より誤差が大きいかどうかで判定するのです。